In [ ]:
import serial
import time
import string
import pynmea2
from pubnub.pnconfiguration import PNConfiguration
from pubnub.pubnub import PubNub
from pubnub.exceptions import PubNubException
pnChannel = "gps_tracking";
pnconfig = PNConfiguration()
pnconfig.subscribe_key = 'sub-c-385aee6a-2d74-4305-91bd-cdc8121481ee'
pnconfig.publish_key = 'pub-c-0293bc60-ff01-4632-929f-51d16ed2b533'
pnconfig.ssl = False
pnconfig.uuid = 'hw'
pubnub = PubNub(pnconfig)
pubnub.subscribe().channels(pnChannel).execute()
while True:
    port="/dev/ttyAMA0"
    ser=serial.Serial(port, baudrate=9600, timeout=0.5)
    dataout = pynmea2.NMEAStreamReader()
    newdata=ser.readline()
    if newdata[0:6] == b"$GPRMC":
        newdata = newdata.decode()
        newmsg=pynmea2.parse(newdata)
        lat=newmsg.latitude
        lng=newmsg.longitude
        print(lat, lng)
        try:
            envelope = pubnub.publish().channel(pnChannel).message({
            'lat':lat,
            'lng':lng
            }).sync()
            print( envelope.result.timetoken)
        except PubNubException as e:
            handle_exception(e)